In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import matplotlib.pyplot as plt
from tqdm import tqdm
%load_ext autoreload
%autoreload 2
import mmm_transformations
import mmm_preprocessing
import mmm_modeling
import mmm_response_curves
import mmm_optimization

# Preprocessing

In [24]:
df = pd.read_csv('bioxcel_pr_definitive_month_level.csv')
df

c:\program files (x86)\python\python37-32\lib\site-packages\IPython\core\interactiveshell.py:3267: DtypeWarning: Columns (23,54,62) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,definitive_id,date,shipped_quantity,speaker_npi_ct,pp_imp_tot,pp_imp_desktop,pp_imp_mobile,pp_imp_set_top_box,pp_imp_tablet,pp_imp_connected_device,...,payor_mix_medicaid_days,payor_mix_private/self-pay/other_days,#_of_discharges,hospital_compare_overall_rating,#_of_staffed_beds,psychiatric_intensive_care_unit_beds,psychiatric_unit_beds,patients_discharged_on_multiple_antipsychotic_medications_with_appropriate_justification_rate,_hours_of_seclusion_rate,hours_of_physical_restraint_use_rate
0,1,2022-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2022-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2022-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2022-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2022-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75707,999006,2023-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,108.0,NaN,NaN,NaN,NaN,NaN
75708,999006,2023-03,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,108.0,NaN,NaN,NaN,NaN,NaN
75709,999006,2023-04,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,108.0,NaN,NaN,NaN,NaN,NaN
75710,999006,2023-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,108.0,NaN,NaN,NaN,NaN,NaN


In [38]:
df_pp = df.copy()
media = ['speaker_npi_ct', 'pp_imp_tot', 'rxnt_imp', 'webmd_imp_tot', 'call_tot']
binary = ['2023_targets']
non_media_cat = ['formulary_type', 'segmentation', 'flag']
non_media_num = [x for x in df_pp.columns if (x.startswith('anti')) | (x.startswith('benzo')) | (x.startswith('payor'))] + ['net_patient_revenue']
cols = ['definitive_id', 'date', 'shipped_quantity'] + media + non_media_cat + binary + non_media_num
df_pp = df_pp[cols]
# filter to only ordering accounts
ordering_acc = df_pp.groupby('definitive_id')['shipped_quantity'].sum().reset_index()
ordering_acc = ordering_acc[ordering_acc['shipped_quantity']>0]['definitive_id'].tolist()
df_pp = df_pp[df_pp['definitive_id'].isin(ordering_acc)]
# replace nulls for media cols and shipped quantity with 0
for i in media + ['shipped_quantity'] + binary:
    df_pp[i] = df_pp[i].fillna(0)
# replace nulls for non media categorical columns with unknown
for i in non_media_cat:
    df_pp[i] = df_pp[i].fillna('Unknown')
# convert payor mix cols to float
for i in non_media_num:
    if i.startswith('payor'):
        df_pp[i] = df_pp[i].str.replace('%','').astype(float)/100
# fillna for non media numeric columns with mean
for i in non_media_num:
    df_pp[i] = df_pp[i].fillna(np.mean(df_pp[i]))
df_pp

,definitive_id,date,shipped_quantity,speaker_npi_ct,pp_imp_tot,rxnt_imp,webmd_imp_tot,call_tot,formulary_type,segmentation,...,benzodiazepine_im_iv_pack_units,benzodiazepine_im_iv_volume_units,benzodiazepine_im_iv_wac_dollars,benzodiazepine_oral_pack_units,benzodiazepine_oral_volume_units,benzodiazepine_oral_wac_dollars,payor_mix_medicare_days,payor_mix_medicaid_days,payor_mix_private/self-pay/other_days,net_patient_revenue
1764,1047935,2022-05,0.0,0.0,0.0,0.0,0.0,0.0,Unknown,Unknown,...,14350.786885,43573.295082,40057.694649,134.0,15806.557377,1616.665,0.255917,0.098887,0.656733,4.679662e+08
1765,1047935,2022-06,0.0,0.0,0.0,0.0,0.0,0.0,Unknown,Unknown,...,14350.786885,43573.295082,40057.694649,134.0,15806.557377,1616.665,0.255917,0.098887,0.656733,4.679662e+08
1766,1047935,2022-07,0.0,0.0,0.0,0.0,0.0,0.0,Unknown,Unknown,...,14350.786885,43573.295082,40057.694649,134.0,15806.557377,1616.665,0.255917,0.098887,0.656733,4.679662e+08
1767,1047935,2022-08,0.0,0.0,0.0,0.0,0.0,0.0,Unknown,Unknown,...,14350.786885,43573.295082,40057.694649,134.0,15806.557377,1616.665,0.255917,0.098887,0.656733,4.679662e+08
1768,1047935,2022-09,0.0,0.0,0.0,0.0,0.0,0.0,Unknown,Unknown,...,14350.786885,43573.295082,40057.694649,134.0,15806.557377,1616.665,0.255917,0.098887,0.656733,4.679662e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,795,2023-02,0.0,0.0,741.0,178.0,0.0,1.0,Unknown,A,...,10281.000000,76510.000000,53004.771600,96.0,10000.000000,839.150,0.150000,0.081000,0.769000,6.970485e+08
69996,795,2023-03,0.0,0.0,947.0,125.0,0.0,4.0,Unknown,A,...,10281.000000,76510.000000,53004.771600,96.0,10000.000000,839.150,0.150000,0.081000,0.769000,6.970485e+08
69997,795,2023-04,0.0,0.0,1362.0,0.0,0.0,1.0,Unknown,A,...,10281.000000,76510.000000,53004.771600,96.0,10000.000000,839.150,0.150000,0.081000,0.769000,6.970485e+08
69998,795,2023-05,0.0,0.0,0.0,0.0,0.0,3.0,Unknown,A,...,10281.000000,76510.000000,53004.771600,96.0,10000.000000,839.150,0.150000,0.081000,0.769000,6.970485e+08


In [28]:
# imputation statistics
imputation = df_pp.groupby('definitive_id')[non_media_num].sum().reset_index()
for i in non_media_num:
    print(f"{i}:{len(imputation[imputation[i]==0])}")

antipsychotic_im_iv_pack_units:8
antipsychotic_im_iv_volume_units:8
antipsychotic_im_iv_wac_dollars:8
antipsychotic_oral_pack_units:8
antipsychotic_oral_volume_units:8
antipsychotic_oral_wac_dollars:8
benzodiazepine_im_iv_pack_units:8
benzodiazepine_im_iv_volume_units:8
benzodiazepine_im_iv_wac_dollars:8
benzodiazepine_oral_pack_units:8
benzodiazepine_oral_volume_units:8
benzodiazepine_oral_wac_dollars:8
payor_mix_medicare_days:10
payor_mix_medicaid_days:17
payor_mix_private/self-pay/other_days:9
net_patient_revenue:12


In [39]:
# one-hot encode non media categorical columns
preprocessing = mmm_preprocessing.MMMPreprocessing()
df_pp = preprocessing.one_hot(df_pp, non_media_cat)
df_pp

,definitive_id,date,shipped_quantity,speaker_npi_ct,pp_imp_tot,rxnt_imp,webmd_imp_tot,call_tot,formulary_type,segmentation,...,formulary_type_On Restricted,formulary_type_Unknown,segmentation_A,segmentation_B,segmentation_C,segmentation_D,segmentation_Unknown,flag_Unknown,flag_Wave I,flag_Wave II
1764,1047935,2022-05,0.0,0.0,0.0,0.0,0.0,0.0,Unknown,Unknown,...,0,1,0,0,0,0,1,1,0,0
1765,1047935,2022-06,0.0,0.0,0.0,0.0,0.0,0.0,Unknown,Unknown,...,0,1,0,0,0,0,1,1,0,0
1766,1047935,2022-07,0.0,0.0,0.0,0.0,0.0,0.0,Unknown,Unknown,...,0,1,0,0,0,0,1,1,0,0
1767,1047935,2022-08,0.0,0.0,0.0,0.0,0.0,0.0,Unknown,Unknown,...,0,1,0,0,0,0,1,1,0,0
1768,1047935,2022-09,0.0,0.0,0.0,0.0,0.0,0.0,Unknown,Unknown,...,0,1,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,795,2023-02,0.0,0.0,741.0,178.0,0.0,1.0,Unknown,A,...,0,1,1,0,0,0,0,0,0,1
69996,795,2023-03,0.0,0.0,947.0,125.0,0.0,4.0,Unknown,A,...,0,1,1,0,0,0,0,0,0,1
69997,795,2023-04,0.0,0.0,1362.0,0.0,0.0,1.0,Unknown,A,...,0,1,1,0,0,0,0,0,0,1
69998,795,2023-05,0.0,0.0,0.0,0.0,0.0,3.0,Unknown,A,...,0,1,1,0,0,0,0,0,0,1


In [40]:
df_pp.columns.tolist()

['definitive_id',
 'date',
 'shipped_quantity',
 'speaker_npi_ct',
 'pp_imp_tot',
 'rxnt_imp',
 'webmd_imp_tot',
 'call_tot',
 'formulary_type',
 'segmentation',
 'flag',
 '2023_targets',
 'antipsychotic_im_iv_pack_units',
 'antipsychotic_im_iv_volume_units',
 'antipsychotic_im_iv_wac_dollars',
 'antipsychotic_oral_pack_units',
 'antipsychotic_oral_volume_units',
 'antipsychotic_oral_wac_dollars',
 'benzodiazepine_im_iv_pack_units',
 'benzodiazepine_im_iv_volume_units',
 'benzodiazepine_im_iv_wac_dollars',
 'benzodiazepine_oral_pack_units',
 'benzodiazepine_oral_volume_units',
 'benzodiazepine_oral_wac_dollars',
 'payor_mix_medicare_days',
 'payor_mix_medicaid_days',
 'payor_mix_private/self-pay/other_days',
 'net_patient_revenue',
 'formulary_type_Accessible/non-formulary',
 'formulary_type_Off Formulary',
 'formulary_type_On Formulary',
 'formulary_type_On Restricted',
 'formulary_type_Unknown',
 'segmentation_A',
 'segmentation_B',
 'segmentation_C',
 'segmentation_D',
 'segmentatio

In [42]:
df_pp.to_csv('bioxcel_pr_time_series_for_modeling.csv', index=False)

# Transformations

In [43]:
transform = mmm_transformations.MMMTransformations()

In [44]:
df_t = transform.lag_dv(df_pp, 'shipped_quantity', 3)
df_t = transform.lag_dv(df_t, 'speaker_npi_ct', 3)
df_t = transform.lag_dv(df_t, 'pp_imp_tot', 3)
df_t = transform.lag_dv(df_t, 'rxnt_imp', 3)
df_t = transform.lag_dv(df_t, 'webmd_imp_tot', 3)
df_t

,definitive_id,date,shipped_quantity,speaker_npi_ct,pp_imp_tot,rxnt_imp,webmd_imp_tot,call_tot,formulary_type,segmentation,...,speaker_npi_ct_lag3,pp_imp_tot_lag1,pp_imp_tot_lag2,pp_imp_tot_lag3,rxnt_imp_lag1,rxnt_imp_lag2,rxnt_imp_lag3,webmd_imp_tot_lag1,webmd_imp_tot_lag2,webmd_imp_tot_lag3
1764,1047935,2022-05,0.0,0.0,0.0,0.0,0.0,0.0,Unknown,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1765,1047935,2022-06,0.0,0.0,0.0,0.0,0.0,0.0,Unknown,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1766,1047935,2022-07,0.0,0.0,0.0,0.0,0.0,0.0,Unknown,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1767,1047935,2022-08,0.0,0.0,0.0,0.0,0.0,0.0,Unknown,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1768,1047935,2022-09,0.0,0.0,0.0,0.0,0.0,0.0,Unknown,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,795,2023-02,0.0,0.0,741.0,178.0,0.0,1.0,Unknown,A,...,0.0,0.0,0.0,0.0,174.0,0.0,0.0,0.0,0.0,0.0
69996,795,2023-03,0.0,0.0,947.0,125.0,0.0,4.0,Unknown,A,...,0.0,741.0,0.0,0.0,178.0,174.0,0.0,0.0,0.0,0.0
69997,795,2023-04,0.0,0.0,1362.0,0.0,0.0,1.0,Unknown,A,...,0.0,947.0,741.0,0.0,125.0,178.0,174.0,0.0,0.0,0.0
69998,795,2023-05,0.0,0.0,0.0,0.0,0.0,3.0,Unknown,A,...,0.0,1362.0,947.0,741.0,0.0,125.0,178.0,0.0,0.0,0.0


In [45]:
df_t.describe()

,definitive_id,shipped_quantity,speaker_npi_ct,pp_imp_tot,rxnt_imp,webmd_imp_tot,call_tot,2023_targets,antipsychotic_im_iv_pack_units,antipsychotic_im_iv_volume_units,...,speaker_npi_ct_lag3,pp_imp_tot_lag1,pp_imp_tot_lag2,pp_imp_tot_lag3,rxnt_imp_lag1,rxnt_imp_lag2,rxnt_imp_lag3,webmd_imp_tot_lag1,webmd_imp_tot_lag2,webmd_imp_tot_lag3
count,9.660000e+02,966.000000,966.000000,966.000000,966.000000,966.000000,966.000000,966.000000,966.000000,966.000000,...,966.000000,966.000000,966.000000,966.000000,966.000000,966.000000,966.000000,966.000000,966.000000,966.000000
mean,8.641964e+04,0.318841,0.011387,107.719462,30.993789,0.162526,2.168737,0.637681,2583.672131,7773.724590,...,0.011387,107.719462,107.719462,106.309524,30.993789,30.993789,30.993789,0.162526,0.162526,0.162526
std,2.083328e+05,1.044004,0.139855,365.072364,123.468290,1.684271,5.040303,0.480919,2218.549180,12566.390419,...,0.139855,365.072364,365.072364,362.846505,123.468290,123.468290,123.468290,1.684271,1.684271,1.684271
min,4.000000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.725000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1210.000000,1417.400000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.784000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2368.000000,3481.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.567000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,3135.000000,7773.724590,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.047935e+06,10.000000,3.000000,2801.000000,1469.000000,28.000000,45.000000,1.000000,10386.000000,79103.000000,...,3.000000,2801.000000,2801.000000,2801.000000,1469.000000,1469.000000,1469.000000,28.000000,28.000000,28.000000


# Final Model Fitting

In [72]:
modeling = mmm_modeling.MMMModeling()

In [130]:
# modeling
channels = ['speaker_npi_ct', 'pp_imp_tot', 'rxnt_imp', 'webmd_imp_tot', 'call_tot']
other = non_media_num + binary + [x for x in df_t.columns if (x.startswith('formulary_type_')) | (x.startswith('segmentation_')) | (x.startswith('flag_'))]
lag_dv = [x for x in df_t.columns if 'shipped_quantity_lag' in x]
lag_channels = [x for x in df_t.columns if ('_lag' in x) & ('shipped_quantity_lag' not in x)]
X = df_t[channels + lag_dv + lag_channels + other]
y = df_t['shipped_quantity']
model = modeling.rf_regressor(df_t, X.columns.tolist(), 'shipped_quantity', 'date')

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_modeling.py:56: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_modeling.py:57: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [131]:
# performance
model['performance']

{'full': {'r2': 0.8597079250146578,
  'rmse': 0.3908354880275674,
  'mape': 0.2892793258632903},
 'train': {'r2': 0.8390833444672158,
  'rmse': 0.3988225055612451,
  'mape': 0.3441958650205634},
 'test': {'r2': -0.4181191385765397,
  'rmse': 1.4136898666254014,
  'mape': 0.6350817007388783}}

In [120]:
# importance
model['importance']

,feature,importance,std
11,pp_imp_tot_lag1,0.113111,0.063965
5,shipped_quantity_lag1,0.087520,0.042524
4,call_tot,0.084541,0.049052
1,pp_imp_tot,0.077458,0.060803
33,payor_mix_medicaid_days,0.069049,0.055503
2,rxnt_imp,0.060891,0.057406
6,shipped_quantity_lag2,0.054836,0.036609
35,net_patient_revenue,0.034888,0.038435
21,antipsychotic_im_iv_volume_units,0.031060,0.029280
20,antipsychotic_im_iv_pack_units,0.027147,0.038842


In [83]:
model['df_preds_test'][['shipped_quantity', 'preds_test']].sort_values(['preds_test'], ascending=False)

,shipped_quantity,preds_test
38203,0.0,4.0925
30055,0.0,3.1900
10399,4.0,2.9200
24819,3.0,2.8305
24568,0.0,2.5500
...,...,...
35923,0.0,0.0000
30057,0.0,0.0000
29273,0.0,0.0000
27635,0.0,0.0000


In [101]:
# scatter plot of predictions
plot_df = pd.DataFrame({'date': model['df_preds_test']['date'], 'preds': model['df_preds_test']['preds_test'],
                        'actual': model['df_preds_test']['shipped_quantity']})
fig = px.scatter(plot_df, x="date", y=plot_df.columns.tolist(), title="Future Hold Out Set")
fig.show()

In [102]:
# scatter plot of predictions
plot_df = pd.DataFrame({'date': model['df_preds_full']['date'], 'preds': model['df_preds_full']['preds_full'],
                        'actual': model['df_preds_full']['shipped_quantity']})
fig = px.scatter(plot_df, x="date", y=plot_df.columns.tolist(), title="Full Data - Model trained on full data")
fig.show()

# Response Curves - Overall

In [132]:
response_curves = mmm_response_curves.MMMResponseCurves()

In [167]:
channels = ['speaker_npi_ct', 'pp_imp_tot', 'rxnt_imp', 'webmd_imp_tot', 'call_tot']
other = non_media_num + binary + [x for x in df_t.columns if (x.startswith('formulary_type_')) | (x.startswith('segmentation_')) | (x.startswith('flag_'))]
lag_dv = [x for x in df_t.columns if 'shipped_quantity_lag' in x]
lag_channels = [x for x in df_t.columns if ('_lag' in x) & ('shipped_quantity_lag' not in x)]
X = df_t[channels + lag_dv + lag_channels + other]

In [164]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
speaker_npi_ct,966.0,1.138716e-02,1.398546e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00
pp_imp_tot,966.0,1.077195e+02,3.650724e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.801000e+03
rxnt_imp,966.0,3.099379e+01,1.234683e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.469000e+03
webmd_imp_tot,966.0,1.625259e-01,1.684271e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.800000e+01
call_tot,966.0,2.168737e+00,5.040303e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,4.500000e+01
shipped_quantity_lag1,966.0,3.188406e-01,1.044004e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+01
shipped_quantity_lag2,966.0,3.188406e-01,1.044004e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+01
shipped_quantity_lag3,966.0,3.188406e-01,1.044004e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+01
speaker_npi_ct_lag1,966.0,1.138716e-02,1.398546e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00
speaker_npi_ct_lag2,966.0,1.138716e-02,1.398546e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00


In [134]:
# overall response curves
channel1 = response_curves.responses(model['full_model'], X, 'speaker_npi_ct', 10, 1)
channel2 = response_curves.responses(model['full_model'], X, 'pp_imp_tot', 5000, 20)
channel3 = response_curves.responses(model['full_model'], X, 'rxnt_imp', 3000, 20)
channel4 = response_curves.responses(model['full_model'], X, 'webmd_imp_tot', 100, 1)
channel5 = response_curves.responses(model['full_model'], X, 'call_tot', 100, 1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 32.84it/s]
D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power

 39%|███████████████████████████████████████████████████▉                                                                                 | 98/251 [00:02<00:03, 39.25it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, 

 47%|██████████████████████████████████████████████████████████████                                                                      | 118/251 [00:03<00:03, 36.57it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 56%|██████████████████████████████████████████████████████████████████████████▏                                                         | 141/251 [00:03<00:02, 39.08it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 65%|█████████████████████████████████████████████████████████████████████████████████████▏                                              | 162/251 [00:04<00:02, 39.30it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 73%|███████████████████████████████████████████████████████████████████████████████████████████████▋                                    | 182/251 [00:04<00:01, 38.83it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\

...

 64%|█████████████████████████████████████████████████████████████████████████████████████▍                                               | 97/151 [00:02<00:01, 40.34it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\

 78%|███████████████████████████████████████████████████████████████████████████████████████████████████████▏                            | 118/151 [00:03<00:00, 36.95it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 92%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌          | 139/151 [00:03<00:00, 38.21it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\

Error - curve_fit failed


 95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍      | 96/101 [00:02<00:00, 35.06it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 99%|██████████████████████████████████████████████████████████████████

In [135]:
response_curves.plot(channel1['resp_df'], 'touches', ['speaker_npi_ct', 'speaker_npi_ct_hill_estimate'])

In [136]:
response_curves.plot(channel2['resp_df'], 'touches', ['pp_imp_tot', 'pp_imp_tot_hill_estimate'])

In [137]:
response_curves.plot(channel3['resp_df'], 'touches', ['rxnt_imp', 'rxnt_imp_hill_estimate'])

In [138]:
response_curves.plot(channel4['resp_df'], 'touches', ['webmd_imp_tot', 'webmd_imp_tot_hill_estimate'])

In [139]:
response_curves.plot(channel5['resp_df'], 'touches', ['call_tot', 'call_tot_hill_estimate'])

In [ ]:
#overall_resp = pd.concat([channel1['resp_df'],
#                          channel2['resp_df'].drop(['touches'], axis=1),
#                          channel3['resp_df'].drop(['touches'], axis=1),
#                          channel4['resp_df'].drop(['touches'], axis=1),
#                          channel5['resp_df'].drop(['touches'], axis=1),
#                          channel6['resp_df'].drop(['touches'], axis=1),
#                          channel7['resp_df'].drop(['touches'], axis=1),
#                          channel8['resp_df'].drop(['touches'], axis=1),
#                          channel9['resp_df'].drop(['touches'], axis=1)], axis=1)
#fig = response_curves.plot(overall_resp, 'touches', channels + ['competitor_sales_B'])
#fig.update_layout(xaxis_title='Weekly Spend', yaxis_title='Revenue Impact')
#fig

In [168]:
# overall response curves for top non-media
#nonmedia1 = response_curves.responses(model['full_model'], X, 'payor_mix_medicaid_days', 1, 0.01)
nonmedia2 = response_curves.responses(model['full_model'], X, 'net_patient_revenue', 1000000000, 2000000)
nonmedia3 = response_curves.responses(model['full_model'], X, 'antipsychotic_im_iv_volume_units', 20000, 200)

 19%|█████████████████████████▊                                                                                                           | 97/501 [00:03<00:15, 26.01it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 24%|███████████████████████████████▎                                                                                                    | 119/501 [00:04<00:13, 27.79it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\

 27%|████████████████████████████████████                                                                                                | 137/501 [00:04<00:14, 25.65it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 32%|█████████████████████████████████████████▋                                                                                          | 158/501 [00:05<00:14, 24.45it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\

 35%|██████████████████████████████████████████████▎                                                                                     | 176/501 [00:06<00:12, 25.94it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\

...

 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎            | 453/501 [00:17<00:01, 34.34it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\

 94%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌       | 473/501 [00:18<00:00, 35.47it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\

 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉  | 493/501 [00:18<00:00, 33.63it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\

Error - curve_fit failed


D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in true_divide

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

invalid value encountered in true_divide



In [170]:
response_curves.plot(nonmedia2['resp_df'], 'touches', ['net_patient_revenue', 'net_patient_revenue_hill_estimate'])

In [171]:
response_curves.plot(nonmedia3['resp_df'], 'touches', ['antipsychotic_im_iv_volume_units', 'antipsychotic_im_iv_volume_units_hill_estimate'])

# Response Curves - Segments

In [153]:
segment = 'segmentation'
channels = ['speaker_npi_ct', 'pp_imp_tot', 'rxnt_imp', 'webmd_imp_tot', 'call_tot']
other = non_media_num + binary + [x for x in df_t.columns if (x.startswith('formulary_type_')) | (x.startswith('segmentation_')) | (x.startswith('flag_'))]
lag_dv = [x for x in df_t.columns if 'shipped_quantity_lag' in x]
lag_channels = [x for x in df_t.columns if ('_lag' in x) & ('shipped_quantity_lag' not in x)]
segments = [x for x in df_t.columns if x.startswith(f"{segment}_")]
X = df_t[channels + lag_dv + lag_channels + other + [segment]]
X[segments] = 0

In [154]:
channel1_segment = response_curves.responses_segment(model['full_model'], X, 'speaker_npi_ct', 10, 1, segment)

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mm

In [155]:
channel1_segment['fig_hill']

In [156]:
channel2_segment = response_curves.responses_segment(model['full_model'], X, 'pp_imp_tot', 5000, 20, segment)

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power



Error - curve_fit failed


D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power



Error - curve_fit failed


D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in true_divide

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

invalid value encountered in true_divide

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power



Error - curve_fit failed


D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in true_divide

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

invalid value encountered in true_divide

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power



Error - curve_fit failed


D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in true_divide

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

invalid value encountered in true_divide

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power



Error - curve_fit failed


In [157]:
channel2_segment['fig_hill']

In [158]:
channel3_segment = response_curves.responses_segment(model['full_model'], X, 'rxnt_imp', 3000, 20, segment)

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power



Error - curve_fit failed


D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power



Error - curve_fit failed


D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power



Error - curve_fit failed


D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power



Error - curve_fit failed


D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power



Error - curve_fit failed


In [159]:
channel3_segment['fig_hill']

In [160]:
channel4_segment = response_curves.responses_segment(model['full_model'], X, 'webmd_imp_tot', 100, 1, segment)

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mm

In [161]:
channel4_segment['fig_hill']

In [162]:
channel5_segment = response_curves.responses_segment(model['full_model'], X, 'call_tot', 100, 1, segment)

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mm

In [163]:
channel5_segment['fig_hill']

# Next Steps

In [172]:
# get remaining financial data
# speaker seems to have high impact but low importance for probably noisy curve
# call curve has high accuracy
# segment B responds best to calls
# check responses for other segments
# breakout calls into type
# consider segment level data sets
# fit ridge regression to get directionality
# try cross sectional data